# Proyecto Final de Organización y Arquitectura de Computadora
## Interrupciones y Control de Procesos en el Desarrollo y Ejecución de Programa
### Ferrer, Sebastián. E-8-211301
### Lucero, Pedro. 8-973-1067

### Plan:
De lo que voy pensando podemos tener
- Interrupción - Clase
- Ejecución - Clase
- anhadir_interrupcion
- ver_detalles (Tipo las puras interrupciones con cada T)
- ver_resultado (Que es la tabla desarrollada)
- ver_bitacora
- (Very loco y solo si hay tiempo) meterle pandas pa csv

In [1]:
class Interrupcion:
    def __init__(self, IRQ, prioridad, funcion):
        self.IRQ = IRQ
        self.prioridad = prioridad
        self.funcion = funcion
    def __repr__(self):
        return f"I({self.funcion})"

In [2]:
def_irqs = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
def_prios = [1, 2, 11, 12, 13, 14, 15, 3, 4, 5, 6, 7, 8, 9, 10]
def_funcs = ["Reloj del sistema", 
             "Teclado",
             "COM2 y COM4",
             "COM1 y COM3",
             "Libre(5)",
             "Controlador Floppy - Diskette",
             "Puerto Paralelo - Impresora",
             "Reloj (tics) en tiempo real CMOS",
             "Red, sonido, puerto SCSI",
             "Libre(10)",
             "Libre(11)",
             "PS-mouse",
             "Co-procesador matemático",
             "Canal IDE primario(Disco)",
             "Libre(15)"]

interrupciones = {irq: Interrupcion(irq, prio, func)
                  for irq, prio, func in zip(def_irqs, def_prios, def_funcs)}

In [14]:
from collections import deque # lo usamos para implementar la pila de interrupciones
class Ejecucion:
    def __init__(self, duracion_programa):
        self.duracion_programa = duracion_programa
        self.interrupciones = [] # este es un arreglo de tuplas para manejar T's con duraciones
        # print(self.duracion_programa)
    
    def anhadir_interrupcion(self, timestamp, irq, duracion): # Añadimos las interrupciones y ordenamos
        # Interrupciones: (Timestamp redondeado a 2 decimales | IRQ de la int. | duración (en seg.) de la int.)
        self.interrupciones.append((round(timestamp, 2), interrupciones[irq], duracion))
        self.interrupciones.sort(key=lambda int: int[0])

    def ver_resultado(self):
        pila = deque() # pila: [prioridad, t_restante]
        pila.append([999, self.duracion_programa, "Programa"])
        
        T = 0
        int_index = 0
        while len(pila) > 0 and int_index < len(self.interrupciones):
#             print(pila)
            actual = pila.pop()
            print(actual[2], end= " ")
            print("T=", T, end=" ")
#             print(actual, "")
            sig_T = self.interrupciones[int_index][0]
            sig_prio = self.interrupciones[int_index][1].prioridad
            sig_name = self.interrupciones[int_index][1].funcion
            sig_t_res = self.interrupciones[int_index][2]
            siguiente = [sig_prio, sig_t_res, sig_name]
            int_index += 1
            
            delta_T = sig_T - T
#             print("delta:", delta_T, end=" ")
            
            if actual[1] - delta_T <= 0:
            #→
                T += actual[1]
                print("T=", T)
                int_index -= 1
                continue
            
            actual[1] -= delta_T
            #→
            T += delta_T
            
            print("T=", T)
            
            # 1. Prio sig < actual → sig. "gana"
            if siguiente[0] < actual[0]:
                pila.append(actual)
                pila.append(siguiente)
            
            # 2. Prio sig > actual → actual "gana"
            if siguiente[0] > actual[0]:
                temp_q = deque()
                temp_q.append(actual)
                while pila[-1][0] < siguiente[0]:
                    temp_q.append(pila.pop())
                temp_q.append(siguiente)
                while temp_q:
                    pila.append(temp_q.pop())
#                 pila.append(siguiente)
#                 pila.append(actual)
            
        
        
        while len(pila) > 0:
            actual = pila.pop()
            print(actual[2], end= " ")
            print("T=", T, end=" ")
            T += actual[1]
            print("T=", T)
            

### Prueba pre-armada #1:
- Duracion de programa = 15
- 5 interrupciones:
  - T=3 | disco | 5seg
  - T=5 | red | 10seg
  - T=10 | impresora | 8seg
  - T=24 | disco | 12seg
  - T=42 | red | 5seg
- T final debe ser de: 55

In [15]:
# Prueba 1 pre armada, con lo definido
prueba1 = Ejecucion(15)
prueba1.anhadir_interrupcion(3, 14, 5)
prueba1.anhadir_interrupcion(5, 9, 10)
prueba1.anhadir_interrupcion(10, 7, 8)
prueba1.anhadir_interrupcion(24, 14, 12)
prueba1.anhadir_interrupcion(42, 9, 5)
prueba1.interrupciones

[(3, I(Canal IDE primario(Disco)), 5),
 (5, I(Red, sonido, puerto SCSI), 10),
 (10, I(Puerto Paralelo - Impresora), 8),
 (24, I(Canal IDE primario(Disco)), 12),
 (42, I(Red, sonido, puerto SCSI), 5)]

In [16]:
prueba1.ver_resultado()

Programa T= 0 T= 3
Canal IDE primario(Disco) T= 3 T= 5
Red, sonido, puerto SCSI T= 5 T= 10
Red, sonido, puerto SCSI T= 10 T= 15
Canal IDE primario(Disco) T= 15 T= 18
Puerto Paralelo - Impresora T= 18 T= 24
Canal IDE primario(Disco) T= 24 T= 36
Puerto Paralelo - Impresora T= 36 T= 38
Programa T= 38 T= 42
Red, sonido, puerto SCSI T= 42 T= 47
Programa T= 47 T= 55


### Prueba pre-armada #2:
- Duracion de programa = 30
- 5 interrupciones:
  - T=6 | COM 1 | 5seg
  - T=15 | impresora | 8seg
  - T=20 | disco | 10seg
  - T=32 | COM 1 | 4seg
  - T=36 | Teclado | 3seg
- T final debe ser de: 60

In [17]:
# Prueba 1 pre armada, con lo definido antes
prueba2 = Ejecucion(30)
prueba2.anhadir_interrupcion(6, 4, 5)
prueba2.anhadir_interrupcion(15, 7, 8)
prueba2.anhadir_interrupcion(20, 14, 10)
prueba2.anhadir_interrupcion(32, 4, 4)
prueba2.anhadir_interrupcion(36, 1, 3)
prueba2.interrupciones

[(6, I(COM1 y COM3), 5),
 (15, I(Puerto Paralelo - Impresora), 8),
 (20, I(Canal IDE primario(Disco)), 10),
 (32, I(COM1 y COM3), 4),
 (36, I(Teclado), 3)]

In [18]:
prueba2.ver_resultado()

Programa T= 0 T= 6
COM1 y COM3 T= 6 T= 11
Programa T= 11 T= 15
Puerto Paralelo - Impresora T= 15 T= 20
Canal IDE primario(Disco) T= 20 T= 30
Puerto Paralelo - Impresora T= 30 T= 32
COM1 y COM3 T= 32 T= 36
Puerto Paralelo - Impresora T= 36 T= 36
Teclado T= 36 T= 39
Puerto Paralelo - Impresora T= 39 T= 40
Programa T= 40 T= 60
